In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.FIM import FIMModel

In [6]:
hparams = {
    'scale':'demo',
    'name':'fim',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'epochs':1,
    'k':-1,
    'select':None,
    'save_step':[0],
    'save_each_epoch':True,
    'train_embedding':True,
    'mode':'train',
    'news_id':False,
    'validate':False,
}


In [7]:
vocab, loaders = prepare(hparams)

[2021-03-15 07:06:41,961] INFO (root) Hyper Parameters are
{'scale': 'demo', 'name': 'fim', 'batch_size': 10, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'filter_num': 150, 'embedding_dim': 300, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'epochs': 1, 'k': -1, 'select': None, 'save_step': [0], 'save_each_epoch': True, 'train_embedding': True, 'mode': 'train', 'news_id': False, 'validate': False}
[2021-03-15 07:06:41,963] INFO (root) preparing dataset...
[2021-03-15 07:06:44,253] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [8]:
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(hparams['device'])

In [9]:
train(fimModel, hparams, loaders, interval=10)

[2021-03-15 07:06:49,799] INFO (root) training...
epoch 1 , step 290 , loss: 1.5413: : 298it [00:33,  8.97it/s]
[2021-03-15 07:07:23,833] INFO (root) saved model of epoch 1
[2021-03-15 07:07:23,869] INFO (root) evaluating...
1815it [00:22, 80.41it/s]
[2021-03-15 07:07:47,408] INFO (root) evaluation results:{'auc': 0.5497, 'mean_mrr': 0.2575, 'ndcg@5': 0.2697, 'ndcg@10': 0.34, 'step': 0}


FIMModel(
  (softmax): Softmax(dim=-1)
  (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
  (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
  (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  (ReLU): ReLU()
  (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (DropOut): Dropout(p=0.2, inplace=False)
  (SeqCNN3D): Sequential(
    (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
  )
  (learningToRank): Linear(in_features=320, out_features=1, bias=True)
)